In [1]:
# run once
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MSIs\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MSIs\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import os
import glob
import string
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, plot_confusion_matrix, precision_score, recall_score,r2_score, mean_squared_error, f1_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.corpus import stopwords

In [3]:
path='dataset/Vietnamese Text Readability Corpus/'

In [4]:
def read_text_files(files):
    
    data=pd.DataFrame()
    texts=[]
    labels=[]

    path=os.path.join(files, '*.txt')

    for txt_file in glob.glob(path):
        
        lbl=txt_file.split('/')[-1].split('\\')[-1].split('_')[0]
        #print(lbl)
        text=open(txt_file,'r', encoding="utf8").read().replace('\n',' ')

        texts.append(text)
        labels.append(lbl)
    data['text']=texts
    data['label']=labels
    return data

In [5]:
df=read_text_files(path)
df

,text,label
0,Khái niệm diễn ngôn trong nghiên cứu văn học h...,difficult
1,Tư tưởng nữ học của Đạm Phương nữ sử Trong bài...,difficult
2,Bí thuật của nghề nghiệp 1 . Người muốn là xạ ...,difficult
3,Vài suy tư về Kafka Tác phẩm của Kafka là một ...,difficult
4,Người kể chuyện tự ý thức trong Nỗi buồn chiến...,difficult
...,...,...
1820,> Tôi là một chú chó ngoan ngoãn như bao chú c...,veryeasy
1821,"Ngày khai trường đã đến , em có biết bao nhiêu...",veryeasy
1822,"Sang mùa đông rồi , trời thật là giá rét , lạn...",veryeasy
1823,Cho ngày hôm nay Có hai ngày trong tuần chúng ...,veryeasy


In [6]:
df.shape

(1825, 2)

In [7]:
df['label'].value_counts()

veryeasy     809
easy         453
difficult    321
medium       242
Name: label, dtype: int64

In [8]:
df['text'] = df['text'].apply(
    lambda x: " ".join(x.lower() for x in x.split()))  # lower case conversion
df['text'] = df['text'].str.replace('[^\w\s]', '')  # getting rid of special characters
df['text'] = df['text'].str.replace('\d+', '')  # removing numeric values from between the words
df['text'] = df['text'].apply(
    lambda x: x.translate(string.digits))  # removing numerical numbers
stop = stopwords.words('english')
df['text'] = df['text'].apply(
    lambda x: " ".join(x for x in x.split() if x not in stop))  # removing stop words

C:\Users\MSIs\AppData\Local\Temp\ipykernel_34868\4095906514.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]', '')  # getting rid of special characters
C:\Users\MSIs\AppData\Local\Temp\ipykernel_34868\4095906514.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '')  # removing numeric values from between the words


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df

,text,label
0,khái niệm diễn ngôn trong nghiên cứu văn học h...,0
1,tư tưởng nữ học của đạm phương nữ sử trong bài...,0
2,bí thuật của nghề nghiệp người muốn là xạ thủ ...,0
3,vài suy tư về kafka tác phẩm của kafka là một ...,0
4,người kể chuyện tự ý thức trong nỗi buồn chiến...,0
...,...,...
1820,tôi là một chú chó ngoan ngoãn như bao chú chó...,3
1821,ngày khai trường đã đến em có biết bao nhiêu t...,3
1822,sang mùa đông rồi trời thật là giá rét lạnh bu...,3
1823,cho ngày hôm nay có hai ngày trong tuần chúng ...,3


In [10]:
le.classes_

array(['difficult', 'easy', 'medium', 'veryeasy'], dtype=object)

In [11]:
word = []
punctuation = set({'.', '?', '!', ';', ':'})
for text in df.text.values:
    word.append([])
    sent = []
    text = text.split('\n')
    for t in text:
        if t != "" and t != '\n':
            t = t.split()
            if t[-1] not in punctuation:
                t.append('.')
            word[-1].extend(t)
df['word'] = word

In [12]:
df

,text,label,word
0,khái niệm diễn ngôn trong nghiên cứu văn học h...,0,"[khái, niệm, diễn, ngôn, trong, nghiên, cứu, v..."
1,tư tưởng nữ học của đạm phương nữ sử trong bài...,0,"[tư, tưởng, nữ, học, của, đạm, phương, nữ, sử,..."
2,bí thuật của nghề nghiệp người muốn là xạ thủ ...,0,"[bí, thuật, của, nghề, nghiệp, người, muốn, là..."
3,vài suy tư về kafka tác phẩm của kafka là một ...,0,"[vài, suy, tư, về, kafka, tác, phẩm, của, kafk..."
4,người kể chuyện tự ý thức trong nỗi buồn chiến...,0,"[người, kể, chuyện, tự, ý, thức, trong, nỗi, b..."
...,...,...,...
1820,tôi là một chú chó ngoan ngoãn như bao chú chó...,3,"[tôi, là, một, chú, chó, ngoan, ngoãn, như, ba..."
1821,ngày khai trường đã đến em có biết bao nhiêu t...,3,"[ngày, khai, trường, đã, đến, em, có, biết, ba..."
1822,sang mùa đông rồi trời thật là giá rét lạnh bu...,3,"[sang, mùa, đông, rồi, trời, thật, là, giá, ré..."
1823,cho ngày hôm nay có hai ngày trong tuần chúng ...,3,"[cho, ngày, hôm, nay, có, hai, ngày, trong, tu..."


In [13]:
punctuation_str = u". , ; : ? !"
special_characters_str = u"~ ` @ # $ % ^ & * ( ) - _ + = { } [ ] \\ | \" \' < > / ‘ ’  “ ” … ° ″ ‰ ‱ ¶ § ‖ © ℗ ® ℠ ™"
indexing_characters_str = u"I. II. III. IV. V. VI. VII. VIII. IX. X. XI. XII. 1. 2. 3. 4. 5. 6. 7. 8. 9. \
                            10. 11. 12. a) b) c) d) e) f) g) h) i) j) k) l) m) n) o) p) q) r) s) t) u) v w) x) y) z)"

In [14]:
punctuation_set = set(punctuation_str.split())
special_character_set = set(special_characters_str.split())
indexing_character_set = set(indexing_characters_str.split())

In [15]:
# 3000 words
most_used_words=[]
#most_used_words = set()
with open('dataset/tan_suat_tu.txt','r',encoding='utf-8') as f:
     for line in f.readlines():
            most_used_words.append(line.split('\t')[0])
print(np.shape(most_used_words))

(3000,)


In [16]:
# 3000 syllables
most_used_syls = []
#most_used_syls = set()
with open('dataset/tan_suat_tieng.txt',encoding="utf-8") as f:
    for line in f.readlines():
        most_used_syls.append(line.split('\t')[0])
        #most_used_syls.add(line.split('\t')[0])
        if len(most_used_syls)==3000:
            break
print(np.shape(most_used_syls))


(3000,)


In [17]:
print(np.shape(most_used_words),np.shape(most_used_syls))

(3000,) (3000,)


In [18]:
#sino_words = set()
sino_words=[]
with open('dataset/ds_han_viet.txt','r',encoding="utf-8") as f:
    for line in f.readlines():
        sino_words.append(line.strip())
        if len(sino_words)==3000:
            break
        #sino_words.add(line.strip())
print(np.shape(sino_words))

(3000,)


In [19]:
text = list(df.text.values)

In [20]:
df

,text,label,word
0,khái niệm diễn ngôn trong nghiên cứu văn học h...,0,"[khái, niệm, diễn, ngôn, trong, nghiên, cứu, v..."
1,tư tưởng nữ học của đạm phương nữ sử trong bài...,0,"[tư, tưởng, nữ, học, của, đạm, phương, nữ, sử,..."
2,bí thuật của nghề nghiệp người muốn là xạ thủ ...,0,"[bí, thuật, của, nghề, nghiệp, người, muốn, là..."
3,vài suy tư về kafka tác phẩm của kafka là một ...,0,"[vài, suy, tư, về, kafka, tác, phẩm, của, kafk..."
4,người kể chuyện tự ý thức trong nỗi buồn chiến...,0,"[người, kể, chuyện, tự, ý, thức, trong, nỗi, b..."
...,...,...,...
1820,tôi là một chú chó ngoan ngoãn như bao chú chó...,3,"[tôi, là, một, chú, chó, ngoan, ngoãn, như, ba..."
1821,ngày khai trường đã đến em có biết bao nhiêu t...,3,"[ngày, khai, trường, đã, đến, em, có, biết, ba..."
1822,sang mùa đông rồi trời thật là giá rét lạnh bu...,3,"[sang, mùa, đông, rồi, trời, thật, là, giá, ré..."
1823,cho ngày hôm nay có hai ngày trong tuần chúng ...,3,"[cho, ngày, hôm, nay, có, hai, ngày, trong, tu..."


In [21]:
#dONE
words_list = []
for words in df.word.astype(object):
    words_list.append([])
    for word in words:
        w = word.lower()
        if not w.isdigit() and w not in punctuation_set and w not in special_character_set and w not in indexing_character_set:
              words_list[-1].append(w)
print(np.shape(words_list))

(1825,)


C:\Users\MSIs\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [22]:
#DONE
syls_list=[]
for words in words_list:
    syls = []
    for word in words:
        for syl in word.split('_'):
            syls.append(syl)
    syls_list.append(syls)
        
print(np.shape(syls_list))
    

(1825,)


In [23]:
import numpy as np

#define matrices
C = np.array([7, 5, 6, 3]).reshape(2, 2)
D = np.array([2, 1, 4, 5, 1, 2]).reshape(2, 3)

#perform matrix multiplication
#C.dot(D)
print(D)


[[2 1 4]
 [5 1 2]]


In [24]:
# Number of Sentences
NSen = np.array([len(t.split('\n')) for t in text], dtype=int)
NSen[:5]

array([1, 1, 1, 1, 1])

In [25]:
# Number of Words
NWo = np.array([len(words) for words in words_list], dtype=int)
NWo[:5]

array([ 6826,  7078,  2962,  1962, 15288])

In [26]:
# Number of Syllables
NSyl = np.array([len(syls) for syls in syls_list], dtype=int)
NSyl[:5]

array([ 6826,  7078,  2962,  1962, 15288])

In [27]:
# Number of Characters
NCha = np.array([sum([len(syl) for syl in syls]) for syls in syls_list], dtype=int)
NCha[:5]

array([23780, 23416,  9744,  6607, 52654])

In [28]:
# Number of Distinct Words
distinct_words = [Counter(words) for words in words_list]
NDWo = np.array([len(w) for w in distinct_words], dtype=int)
NDWo[:5]

array([ 853, 1107,  880,  595, 1390])

In [29]:
# Average Word Length calculated in Syllables
AWLS = np.array(NSyl/NWo, dtype=float)
AWLS[:5]

array([1., 1., 1., 1., 1.])

In [30]:
# print(np.shape(n_difficult),np.shape(NWo))

In [31]:
len(words_list)

1825

In [32]:
len(NWo)

1825

In [33]:
n_difficult=[]
for words in words_list:
    temp=0
    for word in words:
        if word in most_used_words:
            temp +=1
            n_difficult.append(temp)

In [34]:
n_difficult=np.array(n_difficult,dtype=int)
PWD=np.array((1-n_difficult/len(NWo)),dtype=float)
PWD[:5]    

array([0.99945205, 0.99890411, 0.99835616, 0.99780822, 0.99726027])

In [35]:
len(NWo)

1825

In [36]:
n_difficult=[]
for syls in syls_list:
    temp=0
    for syl in syls:
        if syl in most_used_syls:
            temp +=1
        n_difficult.append(temp)

In [37]:
n_difficult=np.array(n_difficult,dtype=int)
PDS=np.array((1-n_difficult/len(NSyl)),dtype=float)
PDS[:5]

array([0.99945205, 0.99890411, 0.99835616, 0.99780822, 0.99726027])

In [38]:
# print(len(PDS),len(PWD),len(PSVW),len(PDSVW))

In [39]:
# Percentage of Sino-Vietnamese Words
sino_viet_words = []
for words in words_list:
    
    temp = []
    for word in words:
        if word in sino_words:
            temp.append(word)

In [40]:
sino_viet_words.append(temp)
NSVW = np.array([len(words) for words in sino_viet_words])
PSVW = np.array(NSVW/NWo, dtype=float)
PSVW[:5]

array([0.00175798, 0.00169539, 0.00405132, 0.00611621, 0.00078493])

In [41]:
# Percentage of Distinct Sino-Vietnamese Words
NDSVW = np.array([len(Counter(words)) for words in sino_viet_words])
PDSVW = np.array(NDSVW/NDWo, dtype=float)
PDSVW[:5]

array([0.010551  , 0.00813008, 0.01022727, 0.01512605, 0.00647482])

In [42]:
df['NSen']=NSen
df['NCha']=NCha
df['AWLS']=AWLS
df['PWD']=PWD[:1825]
df['PDS']=PDS[:1825]
df['PSVW']=PSVW
df['PDSVW']=PDSVW
df.head()


,text,label,word,NSen,NCha,AWLS,PWD,PDS,PSVW,PDSVW
0,khái niệm diễn ngôn trong nghiên cứu văn học h...,0,"[khái, niệm, diễn, ngôn, trong, nghiên, cứu, v...",1,23780,1.0,0.999452,0.999452,0.001758,0.010551
1,tư tưởng nữ học của đạm phương nữ sử trong bài...,0,"[tư, tưởng, nữ, học, của, đạm, phương, nữ, sử,...",1,23416,1.0,0.998904,0.998904,0.001695,0.008130
2,bí thuật của nghề nghiệp người muốn là xạ thủ ...,0,"[bí, thuật, của, nghề, nghiệp, người, muốn, là...",1,9744,1.0,0.998356,0.998356,0.004051,0.010227
3,vài suy tư về kafka tác phẩm của kafka là một ...,0,"[vài, suy, tư, về, kafka, tác, phẩm, của, kafk...",1,6607,1.0,0.997808,0.997808,0.006116,0.015126
4,người kể chuyện tự ý thức trong nỗi buồn chiến...,0,"[người, kể, chuyện, tự, ý, thức, trong, nỗi, b...",1,52654,1.0,0.997260,0.997260,0.000785,0.006475


In [43]:
df.to_csv("feature_df.csv", index=False)